# N-Queen Problem using Evolutionary Algorithms
## Kiarash Mokhtari - 9830032

In [1]:
import random
import time

### 1. First we define Initialize Population function which randomly generates individuals.

In [2]:
def initialize_population(population_size: int, board_size: int) -> list:
    population = []
    for _ in range(population_size):
        individual = [random.randint(0, board_size-1) for _ in range(board_size)]
        population.append(individual)

    return population

In [3]:
# Inorder to test the function we can run below code:
def print_matrix(matrix):
    for row in matrix:
        print(row)
        
population_size = 10
board_size = 8

sample_population = initialize_population(population_size, board_size)
print_matrix(sample_population)

[2, 4, 6, 5, 0, 3, 2, 1]
[5, 6, 4, 3, 6, 6, 1, 3]
[0, 5, 7, 3, 5, 7, 2, 1]
[5, 1, 4, 2, 4, 2, 6, 1]
[1, 4, 3, 6, 1, 7, 4, 7]
[3, 0, 1, 3, 5, 7, 7, 6]
[7, 3, 1, 0, 1, 3, 6, 0]
[2, 2, 4, 2, 7, 3, 1, 3]
[4, 3, 6, 1, 5, 5, 1, 2]
[7, 3, 2, 4, 3, 3, 7, 7]


### 2. we define Evaluate Fitness fucntion which evaluates the fitness of each individual in the population.

In [4]:
def evaluate_fitness(individual):
    conflicts = 0
    size = len(individual)

    for i in range(size):
        for j in range(i+1, size):
            if individual[i] == individual[j] or abs(individual[i] - individual[j]) == abs(i - j):
                conflicts += 1

    fitness = 1 / (conflicts + 1)

    return fitness

In [5]:
# Inorder to test the function we can run below code:
for individual in sample_population:
    print(evaluate_fitness(individual))

0.07142857142857142
0.1
0.14285714285714285
0.1111111111111111
0.1111111111111111
0.1111111111111111
0.14285714285714285
0.16666666666666666
0.1111111111111111
0.07692307692307693


### 3. we define Selection fucntion which selects individuals from the population based on their fitness values. .

In [6]:
# 1 approach
def tournament_selection(population, tournament_size):
    selected = []
    
    for _ in range(len(population)):
        sub_population = random.sample(population, tournament_size)
        winner = max(sub_population, key=evaluate_fitness)
        selected.append(winner)
    
    return selected

### 4. we define Crossover fucntion which performs crossover or recombination between selected individuals to create offspring.


In [7]:
def crossover(parent1, parent2):
    size = len(parent1)
    crossover_point = random.randint(1, size - 1)

    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return crossover_point, child1, child2

In [8]:
# Inorder to test the function we can run below code:
parent1 = [0, 3, 6, 4, 7, 1, 5, 2]
parent2 = [4, 1, 3, 6, 2, 0, 7, 5]

print("Children before Crossover:")
print(parent1)
print(parent2)
print("========================================================")

crossover_point, child1, child2 = crossover(parent1, parent2)
print("Children after Crossover:")
print(child1)
print(child2)
print("Crossover Point: ", crossover_point)

Children before Crossover:
[0, 3, 6, 4, 7, 1, 5, 2]
[4, 1, 3, 6, 2, 0, 7, 5]
Children after Crossover:
[0, 3, 6, 6, 2, 0, 7, 5]
[4, 1, 3, 4, 7, 1, 5, 2]
Crossover Point:  3


### 5. we define Mutation fucntion which adds diversity to the population and helps explore different regions of the solution space.

In [9]:
def mutation(individual, mutation_rate):
    size = len(individual)
    mutated_individual = individual.copy()

    for i in range(size):
        if random.random() < mutation_rate:
            new_position = random.randint(0, size - 1)
            mutated_individual[i] = new_position

    return mutated_individual

In [10]:
# Inorder to test the function we can run below code:
individual = [0, 3, 6, 4, 7, 1, 5, 2]
mutation_rate = 0.1

mutated_individual = mutation(individual, mutation_rate)
print(mutated_individual)

[0, 3, 6, 4, 7, 1, 5, 2]


### 6. we define Replacement fucntion which selects individuals from the parent population and the offspring population to form the next generation.

In [11]:
def replacement(parent_population, offspring_population, population_size):
    combined_population = parent_population + offspring_population

    combined_population.sort(key=evaluate_fitness, reverse=True)

    next_generation = combined_population[:population_size]

    return next_generation

### 7. we define Termination fucntion which determines when to stop the algorithm.

In [12]:
def termination_condition(population, max_fitness):
    for individual in population:
        if evaluate_fitness(individual) >= max_fitness:
            return True
    return False

### 8. Main function tournament_selection function

In [13]:
def solve_n_queen(population_size, board_size, max_generations, mutation_rate):
    population = initialize_population(population_size, board_size)
    current_generation = 0

    while not termination_condition(population, 1.0) and current_generation > max_generations:
        parent_population = tournament_selection(population, 3)
        offspring_population = []

        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(parent_population, 2)
            crossover_point, child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, mutation_rate)
            mutated_child2 = mutation(child2, mutation_rate)
            offspring_population.extend([mutated_child1, mutated_child2])

        population = replacement(parent_population, offspring_population, population_size)
        current_generation += 1

    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)
    return best_individual, best_fitness

In [14]:
def print_individual(individual):
    board_size = len(individual)

    for row in range(board_size):
        line = ""
        for col in range(board_size):
            if individual[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line.strip())
        
# Example for 8-Queen
population_size = 1000
board_size = 8
max_generations = 1000
mutation_rate = 0.1

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("8-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

print("\n")

# Example for 16-Queen
population_size = 200
board_size = 16
max_generations = 2000
mutation_rate = 0.5

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("16-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

8-Queen Problem:
Best Individual: [1, 7, 7, 3, 0, 2, 5, 5]
- Q - - - - - -
- - - - - - - Q
- - - - - - - Q
- - - Q - - - -
Q - - - - - - -
- - Q - - - - -
- - - - - Q - -
- - - - - Q - -
Best Fitness: 0.3333333333333333
Execution Time: 0.025638580322265625 seconds


16-Queen Problem:
Best Individual: [4, 12, 6, 0, 9, 15, 6, 8, 14, 1, 2, 0, 15, 3, 10, 12]
- - - - Q - - - - - - - - - - -
- - - - - - - - - - - - Q - - -
- - - - - - Q - - - - - - - - -
Q - - - - - - - - - - - - - - -
- - - - - - - - - Q - - - - - -
- - - - - - - - - - - - - - - Q
- - - - - - Q - - - - - - - - -
- - - - - - - - Q - - - - - - -
- - - - - - - - - - - - - - Q -
- Q - - - - - - - - - - - - - -
- - Q - - - - - - - - - - - - -
Q - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - - Q
- - - Q - - - - - - - - - - - -
- - - - - - - - - - Q - - - - -
- - - - - - - - - - - - Q - - -
Best Fitness: 0.09090909090909091
Execution Time: 0.011492490768432617 seconds


### Now if we edit the Evaluate fitness function inorder to run the 16-Quenn:

In [15]:
def evaluate_fitness(individual):
    conflicts = 0
    size = len(individual)
    rows = [0] * size
    diagonals1 = [0] * (2 * size - 1)
    diagonals2 = [0] * (2 * size - 1)

    # Only consider the upper half of the board
    half_size = size // 2
    for i in range(half_size):
        queen_col = individual[i]
        rows[queen_col] += 1
        diagonals1[i + queen_col] += 1
        diagonals2[half_size - 1 - i + queen_col] += 1

    # Calculate conflicts
    for i in range(half_size):
        queen_col = individual[i]
        conflicts += rows[queen_col] - 1
        conflicts += diagonals1[i + queen_col] - 1
        conflicts += diagonals2[half_size - 1 - i + queen_col] - 1

    # Double the conflicts for the complete board (considering vertical symmetry)
    conflicts *= 2

    # If the board size is odd, consider the middle column
    if size % 2 != 0:
        middle_col = individual[half_size]
        conflicts += rows[middle_col] - 1
        conflicts += diagonals1[half_size + middle_col] - 1
        conflicts += diagonals2[half_size - 1 - middle_col] - 1

    fitness = 1 / (conflicts + 1)

    return fitness


In [16]:
def print_individual(individual):
    board_size = len(individual)

    for row in range(board_size):
        line = ""
        for col in range(board_size):
            if individual[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line.strip())

In [17]:
def solve_n_queen(population_size, board_size, mutation_rate):
    population = initialize_population(population_size, board_size)
    current_generation = 0

    while not termination_condition(population, 1.0):
        parent_population = tournament_selection(population, 3)
        offspring_population = []

        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(parent_population, 2)
            _, child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, mutation_rate)
            mutated_child2 = mutation(child2, mutation_rate)
            offspring_population.extend([mutated_child1, mutated_child2])

        population = replacement(parent_population, offspring_population, population_size)
        
        current_generation_individual = max(population, key=evaluate_fitness)
        fitness_score = evaluate_fitness(current_generation_individual)
        print("Generation:", current_generation, "Individual:", current_generation_individual, "Fitness score:", fitness_score)
        current_generation += 1

    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)
    print("============================================")
    print(f"Example for {board_size}-Queen:")
    print("Best individual:", best_individual)
    print_individual(best_individual)
    print("Best Fitness score:", best_fitness)


In [20]:
population_size = 10000
board_size = 60
mutation_rate = 0.1

start_time = time.time()
solve_n_queen(population_size, board_size, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("Execution Time:", execution_time, "seconds")

Generation: 0 Individual: [41, 47, 44, 57, 5, 21, 10, 14, 0, 26, 32, 9, 59, 50, 33, 12, 1, 27, 54, 13, 29, 35, 8, 53, 0, 58, 5, 50, 6, 23, 55, 28, 51, 50, 20, 33, 50, 54, 45, 50, 32, 29, 0, 37, 40, 45, 58, 28, 45, 57, 36, 50, 30, 59, 40, 49, 5, 33, 38, 9] Fitness score: 0.058823529411764705
Generation: 1 Individual: [41, 47, 44, 57, 5, 21, 10, 14, 0, 26, 32, 9, 59, 50, 33, 12, 1, 27, 54, 13, 29, 35, 8, 53, 0, 58, 5, 50, 6, 23, 55, 28, 51, 50, 20, 33, 50, 54, 45, 50, 32, 29, 0, 37, 40, 45, 58, 28, 45, 57, 36, 50, 30, 59, 40, 49, 5, 33, 38, 9] Fitness score: 0.058823529411764705
Generation: 2 Individual: [41, 47, 44, 57, 5, 21, 10, 14, 0, 26, 32, 9, 59, 50, 33, 12, 1, 27, 54, 13, 29, 35, 8, 53, 0, 58, 5, 50, 6, 23, 55, 28, 51, 50, 20, 33, 50, 54, 45, 50, 32, 29, 0, 37, 40, 45, 58, 28, 45, 57, 36, 50, 30, 59, 40, 49, 5, 33, 38, 9] Fitness score: 0.058823529411764705
Generation: 3 Individual: [51, 27, 2, 52, 37, 33, 17, 38, 50, 1, 4, 48, 59, 39, 32, 58, 54, 20, 43, 25, 34, 8, 19, 9, 12, 10